In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN

from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import json
import os
import re

import gensim 
from gensim.models import Word2Vec
import gensim.downloader

import pickle 

import nltk
from nltk.tokenize import word_tokenize

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

ModuleNotFoundError: No module named 'skmultilearn'

In [2]:
pip install skmultilearn

ERROR: Could not find a version that satisfies the requirement skmultilearn (from versions: none)
ERROR: No matching distribution found for skmultilearn
Note: you may need to restart the kernel to use updated packages.


In [ ]:
nltk.download('punkt')

In [ ]:
tqdm.pandas()

# Opening Files: 

In [ ]:
rcatt_data = '../src/rcatt_training_data_original.csv'
scraped_data = '../src/training_dataset_full.csv'

In [5]:
# transform into dataframes: 

df_r = pd.read_csv(rcatt_data).reset_index(drop = True)
df_r = df_r[~df_r['Text'].duplicated()]
df_s = pd.read_csv(scraped_data).reset_index(drop = True).rename(columns={'text': 'Text'})

In [6]:
# Converting from string to list using literal_eval:

for col in ['mitre_domain', 'tech_name', 'tech_id', 'tactic_id', 'software_id']:
    df_s[col] = df_s[col].apply(literal_eval)

# Merging Files: 

In [7]:
mlb = MultiLabelBinarizer()
Y_s = mlb.fit_transform(df_s['tactic_id'])
Y_s = pd.DataFrame(Y_s, columns=mlb.classes_)

In [6]:
Y_s

,TA0001,TA0002,TA0003,TA0004,TA0005,TA0006,TA0007,TA0008,TA0009,TA0010,...,TA0102,TA0103,TA0104,TA0105,TA0106,TA0107,TA0108,TA0109,TA0110,TA0111
0,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1516,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1518,0,1,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_r = df_r['Text']
Y_r = df_r[[col for col in df_r.columns if col.startswith('TA')]]

In [10]:
Y_r

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
0,0,1,0,1,1,0,1,0,1,1,1,0
1,0,0,0,0,0,0,1,0,0,0,0,0
2,1,1,0,1,0,0,1,0,1,1,1,0
3,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0,0,0,1,0,0,0,0,0,0,0,0
1486,0,0,0,1,0,0,0,0,0,0,0,0
1487,0,0,0,1,0,0,0,0,0,0,0,0
1488,0,0,0,1,0,0,0,0,0,0,0,0


In [9]:
Y1 = Y_s[Y_r.columns]
Y_s = Y1[Y1.sum(axis=1)>0] 
X_s = df_s['Text']
X_s = X_s[Y1.sum(axis=1)>0] # all urls who map at least one of the tactic in Y1

In [12]:
Y1

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
0,0,0,1,1,1,0,1,0,0,0,0,0
1,0,0,1,0,0,0,1,0,0,0,0,0
2,0,1,1,0,1,0,1,0,1,1,1,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1515,1,1,0,1,1,1,1,0,1,1,1,1
1516,0,1,0,1,1,0,1,0,0,0,1,0
1517,0,0,0,0,0,0,0,0,0,0,1,0
1518,0,1,0,1,0,0,1,0,1,1,1,0


# Pre-processing: 

In [9]:
# Open Pickle: 

with open('tactic_preprocessing.pickle', 'rb') as handle:
    tac_preprocessing = pickle.load(handle)

In [48]:
np.mean(list(map(len, X_r))) # length of text - average 21,542 words 

21542.208588957055

In [13]:
nlp = spacy.load("en_core_web_sm", disable=['ner']) 

In [14]:
# Lemmatisation: 

df = pd.DataFrame({"text": X_r})
df['lemma'] = df['text'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))


KeyboardInterrupt: 

In [100]:
# Stemming: 

# stemmer = SnowballStemmer(language='english')
# df['stemmer'] = df['text'].apply(lambda x: " ".join([stemmer.stem(token) for token in x]))


In [ ]:
# Save as pickle: 

with open('tactic_preprocessing.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Split dataset: 

In [10]:
# change x train to lemma 

X_r_train, X_test_text, Y_r_train, Y_test = train_test_split(X_r, Y_r, test_size=0.3,
                                                    random_state = 10)

In [11]:
X_train_text = pd.concat([X_r_train, X_s])

In [12]:
Y_train = pd.concat([Y_r_train, Y_s]).reset_index(drop=True)

In [16]:
Y_train

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,0
4,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2371,1,1,0,1,1,1,1,0,1,1,1,1
2372,0,1,0,1,1,0,1,0,0,0,1,0
2373,0,0,0,0,0,0,0,0,0,0,1,0
2374,0,1,0,1,0,0,1,0,1,1,1,0


# Feature Extraction:

In [13]:
def feature_extraction(featureExtract, X_train_text, X_test_text, average = False):
    if featureExtract in ['CountVectorizer', 'TfIdfVectorizer']:
        if featureExtract== 'CountVectorizer':
            fe = CountVectorizer(analyzer ='word', stop_words ='english', lowercase = True, min_df = 2, max_df = 0.99) # if words used less than 0.001 % --> ignore  
        else:
            fe = TfidfVectorizer(analyzer = 'word', stop_words='english', lowercase=True, min_df =2, max_df=0.99)
        
        X_train = fe.fit_transform(X_train_text)
        X_train = pd.DataFrame(X_train.toarray(), columns = fe.get_feature_names()) 
        X_test = fe.transform(X_test_text)
        X_test = pd.DataFrame(X_test.toarray(), columns = fe.get_feature_names())
    
    else:
        word2vec = gensim.downloader.load(featureExtract)
        # sent is tokenised sentence on which we do the embedding
        def get_embeddings(sent):
            emb = model[[word for word in sent if word in model.vocab]]
            return np.mean(emb, axis=0) if average else np.sum(emb, exis=0)
        #perform tokenisation
        X_train = X_train_text.progress_apply(nltk.word_tokenize).progress_apply(get_embeddings)
        X_test = X_test_text.progress_apply(nltk.word_tokenize).progress_apply(get_embeddings)
    return X_train, X_test
        
        
        

In [ ]:
#X_train, X_test = feature_extraction('CountVectorizer', X_train_text, X_test_text)

In [ ]:
#X_train, X_test = feature_extraction('TFIDF', X_train_text, X_test_text)

In [ ]:
X_train, X_test = feature_extraction('word2vec-google-news-300', X_train_text, X_test_text)

In [34]:
# ---------------- Word2Vec ---------------------: 

# word2vec = gensim.downloader.load('word2vec-google-news-300')


In [ ]:
# ---------------- Glove ---------------------: 

# load the whole embedding into memory
embeddings_index = dict()
f = open('../data/glove/glove.6B.100d.txt')

# loop through each lines as key, values as coefficients
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs 

f.close()

X_train = df['lemma'].apply(lambda x: [embeddings_index[word] for word in x]) 


In [19]:
X_train.shape

(2376, 74675)

In [20]:
Y_train.shape

(2376, 12)

# Measuring Cosine Similarity to Remove Duplicates: 

In [ ]:
similarities = cosine_similarity(X_train, X_test)

In [ ]:
duplicates = set()
for i in range(similarities.shape[0]):
    for j in range(similarities.shape[1]):
        if similarities[i][j] > 0.9:
            # print(i, j, similarities[i][j])
            duplicates.add(i)

In [ ]:
X_train = X_train[~X_train.index.isin(duplicates)]
Y_train = Y_train[~Y_train.index.isin(duplicates)]

In [80]:
Y_train

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,0
4,0,1,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2371,1,1,0,1,1,1,1,0,1,1,1,1
2372,0,1,0,1,1,0,1,0,0,0,1,0
2373,0,0,0,0,0,0,0,0,0,0,1,0
2374,0,1,0,1,0,0,1,0,1,1,1,0


# Visualisation:

In [ ]:
Y_train.sum(axis=0).sort_values(ascending=False).plot(kind='bar')

# Naive Bayes:

In [ ]:
naive_bayes_classifier = OneVsRestClassifier(MultinomialNB())
naive_bayes_classifier.fit(X_train, Y_train)

In [ ]:
y_pred_proba = pd.DataFrame(naive_bayes_classifier.predict_proba(X_test), columns = Y_test.columns)

In [ ]:
y_pred = (y_pred_proba > 0.005).astype(int) # if increase threshold, recall decreases and precision (could) increase

In [ ]:
precision_score(Y_test, y_pred, average ='macro')

In [ ]:
precision_score(Y_test, y_pred, average ='micro')

In [ ]:
fbeta_score(Y_test, y_pred, beta=0.5, average ='macro')

In [ ]:
fbeta_score(Y_test, y_pred, beta=0.5, average ='micro')

In [ ]:
recall_score(Y_test, y_pred, average='macro')

In [ ]:
recall_score(Y_test, y_pred, average='micro')

# SVC:

In [ ]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


In [ ]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [ ]:
precision_score(Y_test, Y_pred, average ='macro')

In [ ]:
precision_score(Y_test, Y_pred, average ='micro')

In [ ]:
fbeta_score(Y_test, Y_pred, beta=0.5, average ='macro')

In [ ]:
fbeta_score(Y_test, Y_pred, beta=0.5, average ='micro')

In [ ]:
recall_score(Y_test, Y_pred, average='macro')

In [ ]:
recall_score(Y_test, Y_pred, average='micro')

# Multi-label KNN: 

In [ ]:
knn = MLkNN(k = 3)

In [ ]:
# train
knn.fit(X_train.values, Y_train.values)

# predict
predictions = knn.predict(X_test.values)

In [ ]:
fbeta_score(Y_test, predictions, beta=0.5, average ='macro')

In [ ]:
fbeta_score(Y_test, predictions, beta=0.5, average ='micro')

In [ ]:
precision_score(Y_test, predictions, average ='macro')

In [ ]:
precision_score(Y_test, predictions, average ='micro')

In [ ]:
recall_score(Y_test, predictions, average='macro')

In [ ]:
recall_score(Y_test, predictions, average='micro')